<center>
    <img src="https://upload.wikimedia.org/wikipedia/commons/a/a8/%D0%9B%D0%9E%D0%93%D0%9E_%D0%A8%D0%90%D0%94.png" width=500px/>
    <font>Python 2023</font><br/>
    <br/>
    <br/>
    <b style="font-size: 2em">Как писать прикладное API</b><br/>
    <br/>
    <font>Камиль Талипов</font><br/>
</center>

# API: Что это?

Контракт между "сервером" и "клиентом"

Надо договориться:

1. Какие данные передается, что знает "сервер", что знает "клиент"
2. Как данные передаются (протоколол)

Протоколы:

1. HTTP/1.1 (1997)
1. HTTP/2 (2015)
1. HTTP/3 (2022)
1. WebSocket (2011)

...

# HTTP/1.1 Protocol

Основные особенности:
1. Обращение через глаголы - GET, POST, PUT, DELETE, ...
2. Сервер не хранит никакой информации о состоянии клиента между запросами (нет сессий, нет истории операций и т.д.)

Более детально: https://developer.mozilla.org/en-US/docs/Web/HTTP/Overview


<img src="https://drive.google.com/uc?export=view&id=1ioWLhFif--qQiqPmSZ0B8EQmnzaWUmGn" width=900px></img>

# Основные моменты при создании API

О чем надо задумываться при написании API:

1. Формат: REST, RPC и т.д.
2. Технологии (язык, библиотеки и т.д.)
3. Документация
4. Тестирование
5. Валидация прав и данных от пользователя
6. Сообщение об ошибках
7. Расширяемость и версионирование
8. Безопасность
9. Масштабируемость
10. И многое другое...

Попробуем написать небольшое REST API.

Попутно разберем разные библиотеки и обсудим хорошие практики при создании API.

# Часть 0. Особенности REST API

REST API - старый вид протокола. 

Его история начинается с PhD диссертации <a href="https://www.ics.uci.edu/~fielding/pubs/dissertation/top.htm">Roy Thomas Fielding - Architectural Styles and
the Design of Network-based Software Architectures</a>.

REST API - это не какой-то строгий стандарт, а набор принципов. Тут нет одного готового решения. 

REST API основан на идее использования протокола HTTP для создания API.

Принципы:
1. Взаимодействие клиента и сервера происходит по некоторому заданному интерфейсу. При этом они не знают внутреннее устройство друг друга. 
2. Сервер не хранит никакой информации о состоянии клиента между запросами (нет сессий, нет истории операций и т.д.)
3. Все взаимодействие происходит через оперирование ресурсами, которые определяются через URL.
4. Ресурсы имеют четкую структуру.
5. URL - уникальный идентификатор ресурса.
6. Взаимодействие с ресурсами происходит через HTTP методы: `GET`, `POST`, `PUT`, `DELETE`, `PATCH` и т.д.
7. Сервер сообщает, что и как кешировать

Поскольку REST API основан на протоколе HTTP, то основой этого протокола являются две сущности: Запрос и Ответ.

REST API наделяет HTTP-методы некоторый семантикой. `GET` - получить ресурс, `POST` - загрузить ресурс и т.д.
Можно встретить REST API, которые используют только `GET` и `POST`, а есть те которые используют куда больше http методов. 

Тут нет однозначно правильного варианта. Мы обсудим позже особенности использования разных HTTP методов для API.

Дополнительные Keywords:
1. RESTful
2. RESTless
3. HATEOAS

# Часть 2. Фреймворк для API

Для написания REST API существует очень много самых разнообразных библиотек и фреймворков.                           
Вот некоторые из них:
1. AIOHTTP
2. Django, Django Rest
3. Flask, Flask RESTX (ex. Flask-RESTplus), etc
4. Falcon
5. Pyramid
6. FastAPI

Мы остановимся на FastAPI, так как:
1. Поддерживает ассинхронность
2. Легко тестируется
3. Генерирует документацию из коробки

<img src="https://fastapi.tiangolo.com/img/logo-margin/logo-teal.png" width=500px>

# Часть 3. FastAPI

Code demo

Важно! Примеры были с чистым uvicorn.

Он не умеет полноценно перезапускать воркеры.

Для реальных вещей стоит использовать связку Gunicorn + Uvicorn.

Подробнее: https://fastapi.tiangolo.com/deployment/server-workers/

# Часть 4. Еще раз про HTTP

Обращение по HTTP выглядит так:

`VERB /url`

Какие есть HTTP Verbs? Их много:
1. `GET`
2. `POST`
3. `PUT`
4. `DELETE`
5. `PATCH`
6. `HEAD`
7. `OPTIONS`
8. `CONNECT`
9. `TRACE`

Почему так много методов? 

Методы бывают:
* Безопасные и небезопасные
* Идемпотентные и неидемпотентые
* Кешируемые и некешируемые

Безопасный метод - нет side-эффектов. Не меняет состояние ресурса.

Идемпотентный метод - при повторном выполнении того же запроса не изменяет состояние ресурса.

Кешируемый метод - клиент и промежуточные прокси имеют право кешировать результат выполнения запроса.

Что говорит стандарт HTTP/1.1 (RFC 2616) про методы:

`GET`

* Возвращает ресурс (формальнее его представление)
* Должен быть безопасным, идемпотентным, кешируемым

`POST`

* Передает на сервер некоторые данные, связанные с данным ресурсом
* Может создавать новый ресурс с собственным адрессом. Если создается новый ресурс, то в ответе рекомендуется указывать редирект на него.
* Небезопасный, неидемпотентый, некешируемый

`PUT`

* Сохраняет переданный в запросе объект по указанному URL
* Может либо обновлять текущий ресурс, либо создать новый
* Небезопасный, идемпотентый, некешируемый

`DELETE`

* Обратная операция к `PUT`: удаляет ресурс по URL
* Небезопасный, идемпотентный, некешируемый

`PATCH`

* Альтернатива `PUT`: частично модифицирует ресурс
* Можно указывать неполное описание ресурса
* Небезопасный, возможно неидемпотентный, некешируемый

Связь `GET`, `PUT`, `DELETE`.

* Если выполнен `PUT`, то `GET` должен вернуть то же представление
* Если выполнен `DELETE`, то `GET` должен вернуть ошибку

# Часть 4.  API  и HTTP

Как HTTP методы влияют на API? 

Предположим, мы разрабатываем API для отправки денег от одного пользователя другому.

У нас есть GET-метод `/send_money/?from=xxx&to=yyy&count=10`

Хороший ли этот метод?

Что будем если будут перезапросы? Сеть ненадежна и клиент может решить отправить запрос еще раз.

### Случай 1. Перезапросы работают
<img src="https://drive.google.com/uc?export=view&id=1MtMRJXJIeiGYXjqom9Px4dlBeNf8EuTc" width=900px></img>

### Случай 2. Перезапросы не работают
<img src="https://drive.google.com/uc?export=view&id=1rV3nD51gndCbeMf1Cq1ovCx3JK2oXOu6" width=900px></img>

### Случай 3. К чему хотим прийти
<img src="https://drive.google.com/uc?export=view&id=1hpFdmG4UxD_wc2q2m3JUQ5LnFqSAkX9V" width=900px></img>

В нашем API при перезапросах допустима повторная отправка денег. Такое API никуда не годиться!

Что делать? - Использовать HTTP методы правильно.

Можно все сделать через POST запрос. 
Это решение тоже не идеально - мы не можем просто делать перезапросы.

Клиенту нужно убеждаться, что не было уже такого перевода.

Можно сделать API с двумя методами:
1. POST `/create_send_money` - принимает необходимые параметры и возвращает `id` черновика платежа
2. POST `/send_money/?id=xyz` - проводит платеж с заданным `id`.

В таком API мы можем делать ретраи, каждого из двух методов.

Почему второй метод делается через `POST` - потому что он имеет побочные эффекты и мы хотим это подчеркнуть.

Это api тоже не идеально. У нас могут дублироваться `POST` запросы с созданием платежа.

Можно это оставить как есть (и переодически удалять из базы старые платежи).

Альтернатива: заменить первый метод на `POST` с генерацией id на клиенте. <br>
Метод `PUT` как раз позволяет сохранить объект по заданному url. <br>
Значит мы можем на клиенте генерировать `id` и пытаться сохранить платеж на сервере. <br>
В такой схемы мы так же можем ретраить запросы, но нам не нужно будет удалять неудавшиеся платежи


В api Paypal используется похожий подход: https://www.paypal.com/al/smarthelp/article/how-do-i-avoid-duplicate-transactions-ts1097

# Часть 5. Альтернативы: JsonRPC

https://www.jsonrpc.org/specification

# Часть 6. Паджинация

Часто бывает, что данных в ответе очень много и нужно делать паджинацию, то есть разбивку ответа на "страницы"

Что может быть проще: `SELECT * FROM ... LIMIT X, Y`?

Предположим мы выбираем свежие комментарии к посту: <br>
`SELECT * FROM comments ORDER BY date DESC LIMIT 0, 20`.

Все будет работать, если у вас не изменяемый набор объектов. <br>
Но что будет, если комментарии будут добавляться?

Предположим:
1. Пользователь A получил первые 20 комментариев
2. Пользователь B создал комментарий
3. Пользователь A запросил следующие 20 комментариев

Пользователь A увидит дублирующиеся комментарии.



Пологаться на позицию плохо. Нужно что-то, что не меняется с течением времени.

Можно, например, брать время создания комментария.

При запросе следующей страницы передаем время последнего комментария: <br>
`SELECT * FROM comments WHERE date < last_comment_date LIMIT 20`.

Что будет, если будут комментарии с одинаковым временем? Все опять сломается.

Как починить?

Есть несколько способов. Например:
1. Использовать вместо времени `id`. Нужно чтобы были инкрементальное увеличение.
2. Передавать идентификаторы просмотренных комментариев с последним временем
3. Добавить события с тем же временем и фильтровать дубликаты на клиенте

Что насчет перехода в произвольное место?

<img src="https://drive.google.com/uc?export=view&id=1T-mKd1jaI-I_ibdrAfzV54yBEcEZItJs" width=900px></img>

Что можно сделать:
1. Не делать вообще.
2. Переход на последнюю страницу (сделать сортировку в обратном порядке).
3. Нестабильная паджинация (пример Reddit).
4. Что-то сложное для стабильной поджинации.

# Часть 7. Советы по написанию API

## Совет 1. Устройство URL

* Группировка URL по namespace, чтобы избежать коллизий (`/api/core`, `/api/staff/` и т.д.)
* Делать URL для коллекции ресурсов (во множественном числе) (`/api/core/peoples`)
* URL-коллекции + id = URL-ресурса (`/api/core/peoples/{people_id}` или `/api/core/peoples/?id={people_id}`)


Как выбрать способо передачи параметров: через часть url (`/xxx/`) или через query-параметр (`/?id=xxx`)?

* Фильтрацию или поиск лучше делать через query-параметр (так как тут нет какой-либо иерархии объектов)
* Извлечение и манипуляция с объектом лучше делать через часть url

## Совет 2. Использование HTTP методов
Рекомендуется использовать не только `GET` и `POST`.
Это позволяет лучше подчеркнуть безопасность и иденпотентность вызовов.

Можно использовать такую семантику:
* `GET` = Read
* `POST` = Create
* `PUT` = Save
* `PATCH` = Update
* `DELETE` = Delete

* `GET` - безопасный, можно делать ретраи не задумываясь
* `GET`, `PUT`, `DELETE` - идемпотентные. Если что-то сломалось, можно поретраить
* `POST`, `PATCH` - опасные, могут что-то поменять. Надо задуматься перед повторением.

Обычно колекции поддерживают семантику Create и Read.

Ресурсы поддерживают семантику Read, Save, Update, Delete

Что делать если операция сложная? Например копирование ресурса.

Можно делать POST-запрос такого вида:
`POST /api/resources/copy?src_path={src_path}&dst_path={dst_path}`

Некоторые клиенты умеет делать только `GET` и `POST`.

К счастью в HTTP есть заголовок `X-HTTP-Method-Override` для POST-метода, который позволяет указать какой реально метод мы хотим использовать.

## Совет 3. Детали об ошибке

<img src="https://drive.google.com/uc?export=view&id=11ckjLhTmoONpUcTHS2_FXvR5gOy5f5Bd" width=900px></img>

Главное давать подробности ошибки в теле ответа.

Какой status code использовать - открытый вопрос.

Если хочется в логах быстро находить ошибки, то лучше использовать стандартный HTTP-код.

## Совет 4. Не доверяете никому

Проверяйте все данные, которые принимаете.

## Полезные ссылки
1. https://cloud.google.com/apis/design
2. https://github.com/microsoft/api-guidelines
3. https://learn.microsoft.com/en-us/azure/architecture/best-practices/api-design
4. https://geemus.gitbooks.io/http-api-design/content/en/


# Спасибо за внимание!